In [2]:
from numpy import array
from pickle import dump
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [4]:
import pandas as pd
articles = pd.read_csv('../data/arxiv_math_2007.csv')

In [8]:
raw_text = articles.loc[:100, 'abstract'].sum()
lines = raw_text.split('\n')

In [39]:
# load text
raw_text = articles.loc[:100, 'abstract'].sum()
#print(raw_text)

def save_doc(lines, filename):
	data = '\n'.join(lines)
	file = open(filename, 'w')
	file.write(data)
	file.close()

# clean
tokens = raw_text.split()
raw_text = ' '.join(tokens)

# organize into sequences of characters
length = 10
sequences = list()
for i in range(length, len(raw_text)):
	# select sequence of tokens
	seq = raw_text[i-length:i+1]
	# store
	sequences.append(seq)
print('Total Sequences: %d' % len(sequences))

# save sequences to file
out_filename = 'char_sequences.txt'
save_doc(sequences, out_filename)

Total Sequences: 58878


In [40]:
from numpy import array
from pickle import dump
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# load
in_filename = 'char_sequences.txt'
raw_text = load_doc(in_filename)
lines = raw_text.split('\n')

In [41]:
# integer encode sequences of characters
chars = sorted(list(set(raw_text)))
mapping = dict((c, i) for i, c in enumerate(chars))
sequences = list()
for line in lines:
	# integer encode line
	encoded_seq = [mapping[char] for char in line]
	# store
	sequences.append(encoded_seq)

# vocabulary size
vocab_size = len(mapping)
print('Vocabulary Size: %d' % vocab_size)

Vocabulary Size: 91


In [42]:
# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
sequences = [to_categorical(x, num_classes=vocab_size) for x in X]
X = array(sequences)
y = to_categorical(y, num_classes=vocab_size)

In [43]:
# define model
model = Sequential()
model.add(LSTM(75, input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 75)                50100     
_________________________________________________________________
dense_1 (Dense)              (None, 91)                6916      
Total params: 57,016
Trainable params: 57,016
Non-trainable params: 0
_________________________________________________________________
None


In [70]:
# fit model
model.fit(X, y, epochs=50)

# save the model to file
model.save('model.h5')
# save the mapping
dump(mapping, open('mapping.pkl', 'wb'))

Epoch 1/50
58878/58878 [==============================] - 23s 398us/step - loss: 1.5391 - acc: 0.5652
Epoch 2/50
58878/58878 [==============================] - 23s 390us/step - loss: 1.4958 - acc: 0.5772
Epoch 3/50
58878/58878 [==============================] - 21s 353us/step - loss: 1.4575 - acc: 0.5876
Epoch 4/50
58878/58878 [==============================] - 21s 358us/step - loss: 1.4216 - acc: 0.5968
Epoch 5/50
58878/58878 [==============================] - 22s 378us/step - loss: 1.3903 - acc: 0.6060
Epoch 6/50
58878/58878 [==============================] - 22s 372us/step - loss: 1.3595 - acc: 0.6138
Epoch 7/50
58878/58878 [==============================] - 25s 429us/step - loss: 1.3315 - acc: 0.6213
Epoch 8/50
58878/58878 [==============================] - 22s 370us/step - loss: 1.3036 - acc: 0.6294
Epoch 9/50
58878/58878 [==============================] - 22s 371us/step - loss: 1.2785 - acc: 0.6370
Epoch 10/50
58878/58878 [==============================] - 22s 371us/step - loss: 

In [71]:
from pickle import load
from keras.models import load_model
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences

# generate a sequence of characters with a language model
def generate_seq(model, mapping, seq_length, seed_text, n_chars):
    in_text = seed_text
    # generate a fixed number of characters
    for _ in range(n_chars):
        # encode the characters as integers
        encoded = [mapping[char] for char in in_text]
        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        # one hot encode
        encoded = to_categorical(encoded, num_classes=len(mapping))
        #return encoded
        #encoded = encoded.reshape(1, encoded.shape[0], encoded.shape[1])
        # predict character
        yhat = model.predict_classes(encoded, verbose=0)
        # reverse map integer to character
        out_char = ''
        for char, index in mapping.items():
            if index == yhat:
                out_char = char
                break
        # append to input
        in_text += char
    return in_text

# load the model
model = load_model('model.h5')
# load the mapping
mapping = load(open('mapping.pkl', 'rb'))

In [79]:
# test start of rhyme
generate_seq(model, mapping, 10, 'exponential', 20)


'exponential the notion of the s'